# Experimentos

Experimentos são a principal forma de agrupar 3 entidades:

1. **Datasets**: conjunto de textos acompanhados de um *target*, representando um problema de classificação ou regressão;
2. **Pipelines**: são capazes de receber um ou mais textos como entrada e produzir uma saída para o problema de classificação/regressão;
3. **Métricas**: permitem avaliar o desempenho de uma ou mais pipelines para o problema em questão;

A biblioteca `aibox-nlp` disponibiliza diferentes métodos para construção e execução de experimentos em seu pacote `aibox.nlp.experiments`. É possível construir um experimento instanciando cada um dos componentes individualmente ou através do pacote `aibox.nlp.factory`, que possui facilidades para obter as diversas classes da biblioteca.

In [ ]:
import json

import pandas as pd

from aibox.nlp import serialization
from aibox.nlp.factory.experiment import SimpleExperimentBuilder

## Exemplo 1 - Construindo um Experimento

In [ ]:
# === Construindo um experimento para classificação no Essay-BR estendido ===
# Por simplicidade, vamos instanciar um experimento
#   para comparar algumas abordagens para classificação
#   da competência 1 do dataset Essay-BR.
builder = SimpleExperimentBuilder()

# Inicialmente, vamos definir o dataset
builder.dataset('essayBR',
                extended=True,
                target_competence='C1')

# Também é possível passar uma instância
#   de um Dataset diretamente:
# builder.custom_dataset(ds)

# Vamos definir o tipo do problema
builder.classification()

# Vamos definir a seed randômica
builder.seed(42)

# Depois, vamos definir algumas métricas
#   que devem ser calculadas
builder.add_metric('precision', average='weighted')
builder.add_metric('recall', average='weighted')
builder.add_metric('f1', average='weighted')
builder.add_metric('kappa')
builder.add_metric('neighborKappa')

# Depois, vamos definir qual a métrica
#   que deve ser utilizar para escolher a
#   melhor pipeline
builder.best_criteria('precision', average='weighted')

# Agora, vamos adicionar algumas pipelines baseadas
#   em extração de característica
builder.add_feature_pipeline(
    features=['readabilityBR',
              'regencyBR',
              'syntacticComplexityBR',
              'textualSimplicityBR'],
    estimators=['svm',
                'etreesClf',
                'lgbmClf',
                'xgbClf'],
    names=['svm+features',
           'etrees+features',
           'lgbm+features',
           'xgb+features'])

# Agora, vamos adicionar algumas pipelines baseadas
#   em outras estratégias de vetorização
builder.add_vectorizer_pipeline('tfidfVectorizer',
                                estimators=['etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['etrees+tfidf',
                                       'lgbm+tfidf',
                                       'xgb+tfidf'],
                                estimators_configs=[dict(n_estimators=20),
                                                    dict(n_estimators=20),
                                                    dict(n_estimators=20)])
builder.add_vectorizer_pipeline('bertVectorizer',
                                estimators=['svm',
                                            'etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['svm+bert',
                                       'etrees+bert',
                                       'lgbm+bert',
                                       'xgb+bert'])

# Vamos aproveitar um conjunto de características
#   pré-extraídos se ele existir:
features = None
try:
    features = pd.read_csv('essay_br_extended_features.csv')
except Exception:
    ...

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build(features_df=features)

In [ ]:
# === Executando o experimento ===
result = experiment.run()

In [ ]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
result.best_pipeline.name

In [ ]:
# Também podemos listar o valor das melhores métricas:
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

In [ ]:
# Também podemos listar o valor de todas as métricas calculadas:
print(json.dumps({k: {k_: v_.tolist() for k_, v_ in v.items()} 
                  for k, v in result.metrics_history.items()},
                 indent=2,
                 ensure_ascii=False))

In [ ]:
# Por último, também é possível recuperar um DataFrame com as características
#   extraídas para os textos do dataset.
result.extras.df_features.head(3)

In [ ]:
# Podemos salvar a extração de características 
#   para reutiliza rem outro momento.
from pathlib import Path

output = Path('essay_br_extended_features.csv')

if not output.exists():
    result.extras.df_features.to_csv(output,
                                     index=False)

In [ ]:
# Também podemos realizar a serialização
#   da melhor pipeline
pipeline = result.best_pipeline
serialization.save_pipeline(pipeline, f'{pipeline.name}.joblib')

In [ ]:
# Depois, podemos recuperar essa pipeline
#   e utilizá-la para novas predições
pipeline = serialization.load_pipeline(f'{pipeline.name}.joblib')
predictions = pipeline.predict(result.test_df.text.to_numpy())
assert (predictions == result.best_pipeline_test_predictions).all()

## Exemplo 2 - Utilizando Experimentos Pré-Inicializados

O pacote `aibox.nlp.factory` também oferece experimentos pré-inicializados para serem utilizados como baselines. Essencialmente, são utilizadas as configurações padrão para todos os estimadores, vetorizados e extratores de característica.


In [ ]:
# === Construindo um experimento para classificação no Essay-BR clássico ===

# Inicializando um experimento baseado em características com o uso
#   de estimadores clássicos (SVM, Random Forest, XGBoost, etc) e
#   métricas padrão para o problema (precision, recall, f1-score, kappa ou 
#   RMSE, MAE, R2 e MSE)
builder = SimpleExperimentBuilder.features_experiment(42, 'classification')

# Depois, só precisamos definir o dataset
builder.dataset('essayBR',
                extended=False,
                target_competence='C1')

# Vamos aproveitar um conjunto de características
#   pré-extraídos se ele existir:
features = None
try:
    features = pd.read_csv('essay_br_classic_features.csv')
except Exception:
    ...

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build(features_df=features)

In [ ]:
# === Executando o experimento ===
# A extração de todas as características
#   para o dataset pode demorar
#   consideravelmente (2h+)
result = experiment.run()

In [ ]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
print('Melhor pipeline:', result.best_pipeline.name)

# Também podemos obter as métricas
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

In [ ]:
# Podemos salvar a extração de características
# para reutilizar em outro momento.
from pathlib import Path

output = Path('essay_br_classic_features.csv')

if not output.exists():
    result.extras.df_features.to_csv(output,
                                     index=False)